In [4]:
import findspark
#useful to install this tool to simplify spark import
findspark.init()
from pyspark import  SparkContext
sc = SparkContext( 'local', 'pyspark')

In [23]:
import numpy
rdd=sc.textFile('train_no_header.csv')

#obtaining rdd: itemid,rating (users are not useful for this recommendation)
def firstMap (line):
    x=line.split(',')
    return x[1],int(x[2])
rdd = rdd.map(firstMap).cache()

#computing average vote of the entire dataset
totalVotes = rdd.map(lambda x: x[1]).reduce(lambda x,y: x+y)
avgVote = float(totalVotes)/rdd.count()

#creating rdd: itemid, [rating_list]
itemVotes=rdd.groupByKey()
itemVotes=itemVotes.map(lambda x:(x[0],list(x[1])))
itemAverage=itemVotes.map(lambda x: (x[0],float(sum(x[1]))/len(x[1]),len(x[1]))).cache()

#computing average number of vote for item of the entire dataset
avgVoteNumberPerItem=rdd.count()/itemVotes.count()


5


[(u'33475', 9.559667298478576, 72),
 (u'33173', 9.285341399868079, 125),
 (u'32347', 9.204266405548555, 64),
 (u'29848', 9.163152676218388, 63),
 (u'15743', 9.151179774785628, 75)]

In [32]:
#Computing ranking value according to:
#https://www.quora.com/How-is-a-Movies-weighted-rating-calculated-on-IMDb
minimumVotesRequired=avgVoteNumberPerItem*1.5

def tenStarScore (x):
    R=x[1]
    v=x[2]
    m=minimumVotesRequired
    C=avgVote
    return x[0],(v*R+m*C)/(v+m),x[2]
    
itemRank=itemAverage.map(tenStarScore)            

print minimumVotesRequired
itemRank.takeOrdered(10,key=lambda x: -x[1])

7.5


[(u'33475', 9.4734789053368, 72),
 (u'33173', 9.238804324334154, 125),
 (u'32347', 9.120861160479377, 64),
 (u'15743', 9.080503914839703, 75),
 (u'29848', 9.08002231169185, 63),
 (u'36872', 9.054929857683733, 77),
 (u'3431', 9.050931090549307, 64),
 (u'34517', 9.046389464640452, 71),
 (u'29915', 9.038563109113511, 61),
 (u'13502', 9.024319599618897, 60)]